In [47]:
import pandas as pd
import glob
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import manipulating_data as md
from scipy.stats import iqr
import numpy as np
from dateutil import parser
from sklearn.linear_model import LinearRegression

In [48]:
path = glob.glob('**/*dengue_features_train.csv', recursive=True) [0]

In [49]:
deng_train = pd.read_csv(path)

In [50]:
deng_train.head()

city  year  weekofyear week_start_date   ndvi_ne   ndvi_nw   ndvi_se  \
0   sj  1990          18      1990-04-30  0.122600  0.103725  0.198483   
1   sj  1990          19      1990-05-07  0.169900  0.142175  0.162357   
2   sj  1990          20      1990-05-14  0.032250  0.172967  0.157200   
3   sj  1990          21      1990-05-21  0.128633  0.245067  0.227557   
4   sj  1990          22      1990-05-28  0.196200  0.262200  0.251200   

    ndvi_sw  precipitation_amt_mm  reanalysis_air_temp_k  ...  \
0  0.177617                 12.42             297.572857  ...   
1  0.155486                 22.82             298.211429  ...   
2  0.170843                 34.54             298.781429  ...   
3  0.235886                 15.36             298.987143  ...   
4  0.247340                  7.52             299.518571  ...   

   reanalysis_precip_amt_kg_per_m2  reanalysis_relative_humidity_percent  \
0                            32.00                             73.365714   
1                            17.94                             77.368571   
2                            26.10                             82.052857   
3                            13.90                             80.337143   
4                            12.20                             80.460000   

   reanalysis_sat_precip_amt_mm  reanalysis_specific_humidity_g_per_kg  \
0                         12.42                              14.012857   
1                         22.82                              15.372857   
2                         34.54                              16.848571   
3                         15.36                              16.672857   
4                          7.52                              17.210000   

   reanalysis_tdtr_k  station_avg_temp_c  station_diur_temp_rng_c  \
0           2.628571           25.442857                 6.900000   
1           2.371429           26.714286                 6.371429   
2           2.300000           26.714286                 6.485714   
3           2.428571           27.471429                 6.771429   
4           3.014286           28.942857                 9.371429   

   station_max_temp_c  station_min_temp_c  station_precip_mm  
0                29.4                20.0               16.0  
1                31.7                22.2                8.6  
2                32.2                22.8               41.4  
3                33.3                23.3                4.0  
4                35.0                23.9                5.8  

[5 rows x 24 columns]

In [51]:
deng_train.loc[deng_train.ndvi_sw == 0.177617, ]

Empty DataFrame
Columns: [city, year, weekofyear, week_start_date, ndvi_ne, ndvi_nw, ndvi_se, ndvi_sw, precipitation_amt_mm, reanalysis_air_temp_k, reanalysis_avg_temp_k, reanalysis_dew_point_temp_k, reanalysis_max_air_temp_k, reanalysis_min_air_temp_k, reanalysis_precip_amt_kg_per_m2, reanalysis_relative_humidity_percent, reanalysis_sat_precip_amt_mm, reanalysis_specific_humidity_g_per_kg, reanalysis_tdtr_k, station_avg_temp_c, station_diur_temp_rng_c, station_max_temp_c, station_min_temp_c, station_precip_mm]
Index: []

[0 rows x 24 columns]

In [52]:
deng_train['ndvi_se'].describe()

count    1434.000000
mean        0.203783
std         0.073860
min        -0.015533
25%         0.155087
50%         0.196050
75%         0.248846
max         0.538314
Name: ndvi_se, dtype: float64

# Zadanie 1

Celem tej instrukcji jest stworzenie klasy DataSet, która służy do reprezentowania danych tabelarycznych, użytecznych w modelowaniu predykcyjnym.

W ciele klasy DataSet, zdefiniuj metodę __init__, która inicjalizuje nową instancję klasy. Ta metoda powinna przyjmować argumenty path oraz data, z domyślnymi wartościami None. W ciele tej metody użyj instrukcji warunkowych, aby obsłużyć różne przypadki przekazania argumentów:

- Jeżeli podano tylko data, zapisz data jako atrybut instancji.
- Jeżeli podano tylko path, zapisz path jako atrybut instancji i wczytaj dane do atrybutu data za pomocą pd.read_csv(path).
- Jeżeli podano oba, wypisz wartość path oraz typ data i zgłoś wyjątek ValueError.
- W przypadku gdy nie podano żadnego z argumentów wypisz "Musimy skądś wziąć dane!" i zgłoś ValueError z komunikatem "Either path or data should be given." (lub podobnym).

Pamiętaj, jakie domyślne wartości mają argumenty konstruktora i że jeśli nie podamy któregoś argumentu, to będzie miał domyślną wartość.
To pomoże Ci ustalić, czy podano jeden argument, czy obydwa itd.

In [53]:
class DataSet() :

    """ 
    A class used to represents tabular data.
    which can be used in down-stream predictive modelling.

    Attributes
    ----------
    path : str
        The path to the data file.
    data : pandas DataFrame.
        The data loaded from the given path.

    """
    def __init__(self, path = None, data=None) :

        """ 
        Loads the data from the given path
        and stores path and data as instance attributes.
        The data attribute is of type pandas DataFrame.

        Parameters
        ----------
        path  : str, optional. Defaults to None.
            The path to the data file.
        data : pandas DataFrame, optional. Defaults to None.
            The data already in DataFrame form.
        """
        if (data is not None) and (path is None):
            self.data = data
        elif (data is None) and (path is not None):
            self.path = path
            self.data = pd.read_csv(path)
        else :
            print(f'The path is {path}')
            print(f'The data is {repr(data)}')
            raise ValueError("Either path or data should be given, but not both.")

In [54]:
data = DataSet(data = pd.DataFrame([1, 2, 3]))
data.data.head()

0
0  1
1  2
2  3

# Zadanie 1.5 - zadanie na 5 dla ambitnych

In [55]:
class CategoricalVariable() :

    """ 
    A class used to represent a categorical variable from some dataset.

    Attributes
    ----------
    column : pandas Series.
        A column from a pandas DataFrame or a standalone column.
    
    Methods
    -------
    ordinal_encode(column: pd.Series, show_mapping=False) -> pd.Series
        Encodes the given column using ordinal encoding.
    
    one_hot_encode(column: pd.Series, show_mapping=False) -> pd.DataFrame
        Encodes the given column using one-hot encoding.
    """


    def __init__(self, column : pd.Series) :
        self.column = column
    

        
    @staticmethod
    def ordinal_encode(column: pd.Series, show_mapping=False) -> pd.Series:
        encoder = OrdinalEncoder()
        encoder_fitted = encoder.fit(pd.DataFrame(column))
        encoded_data = encoder_fitted.transform(pd.DataFrame(column))
        inverse_transformation = encoder_fitted.inverse_transform(encoded_data)
            
        if show_mapping:
            values_mapping = { e.tolist()[0] : t.tolist() for t, e in\
                                        zip(encoded_data, inverse_transformation) }
            return values_mapping
            
        return pd.Series(encoded_data.flatten(), index=column.index, name=column.name)
        

    @staticmethod
    def one_hot_encode(column: pd.Series, show_mapping = False) -> pd.DataFrame:

        encoder = OneHotEncoder(sparse=False)
        fitted_encoder = encoder.fit(pd.DataFrame(column))
        transformed = fitted_encoder.transform(pd.DataFrame(column))
        inverse_transformation = encoder.inverse_transform(transformed)

        if show_mapping :

            values_to_representations = { i.tolist()[0] : t.tolist() for t, i in\
                                        zip(transformed, inverse_transformation) }
            
            return values_to_representations
        
        
        return pd.DataFrame(transformed, columns=fitted_encoder.get_feature_names_out())

    
    def encode_data(self, method, show_mapping=False) -> pd.DataFrame:
        if method == 'ordinal':
            encoded_df = CategoricalVariable.ordinal_encode(self.column, show_mapping=show_mapping)
        elif method == 'one_hot':
            encoded_df = CategoricalVariable.one_hot_encode(self.column, show_mapping=show_mapping)
        else:
            raise ValueError(f"Encoding method {method} not recognized.")
        return encoded_df 



In [56]:
categorical_col = CategoricalVariable(deng_train['city'])

In [57]:
categorical_col.column

0       sj
1       sj
2       sj
3       sj
4       sj
        ..
1451    iq
1452    iq
1453    iq
1454    iq
1455    iq
Name: city, Length: 1456, dtype: object

In [58]:
categorical_col.encode_data('one_hot')

city_iq  city_sj
0         0.0      1.0
1         0.0      1.0
2         0.0      1.0
3         0.0      1.0
4         0.0      1.0
...       ...      ...
1451      1.0      0.0
1452      1.0      0.0
1453      1.0      0.0
1454      1.0      0.0
1455      1.0      0.0

[1456 rows x 2 columns]

In [59]:
encoded_data = categorical_col.encode_data('ordinal', show_mapping = True)

In [60]:
encoded_data

{'sj': [1.0], 'iq': [0.0]}

In [61]:
categorical_col.encode_data('ordinal')

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
1451    0.0
1452    0.0
1453    0.0
1454    0.0
1455    0.0
Name: city, Length: 1456, dtype: float64

# Zadanie 2

### Instrukcja do napisania 2 metod wewnątrz klasy DateVariable

Klasa DateVariable służy do reprezentowania zmiennej daty z jakiegoś zbioru danych. Będziecie tworzyć dwie metody dla tej klasy: parse_string_date i encode_as_number. Oto jak to zrobić:
1. Metoda parse_string_date(string_date: str) -> pd.Timestamp:
Ta statyczna metoda ma za zadanie przekształcić datę w formacie ciągu znaków na obiekt pd.Timestamp (znacznik czasu pandas). Oto jakie kroki należy wykonać:

- a) Użyj metody pd.to_datetime() z biblioteki pandas, aby przekonwertować string_date na obiekt Timestamp.

- b) Zamień przekonwertowaną datę na standardowy format daty, np. 'YYYY-MM-DD', używając metody strftime('%Y-%m-%d').

- c) Zwróć przekształconą datę jako obiekt Timestamp.

Przykład szablonu metody:

In [62]:
# tu nic nie robisz, masz to zrobić wewnątrz klasy DateVariable, która jest nieco niżej

@staticmethod
def parse_string_date(string_date: str) -> pd.Timestamp:
    # Tu wstaw logikę przekształcania ciągu znaków na Timestamp.
    return # przekształcony Timestamp


2. Metoda encode_as_number() -> pd.Series:
Ta metoda ma za zadanie zakodować datę jako liczbę, gdzie każdy dzień jest reprezentowany przez unikalną liczbę. Realizacja tej metody powinna obejmować następujące kroki:

- a) Sprawdź, czy typ danych w self.column to 'datetime64[ns]'. Jeśli nie, użyj metody apply z biblioteki pandas na self.column, aby zamienić każdą datę (zakładając, że są one w formacie ciągu znaków) na format Timestamp, używając wcześniej napisanej metody statycznej parse_string_date.

- b) Po upewnieniu się, że wszystkie daty są w formacie Timestamp, wyodrębnij z nich rok, miesiąc i dzień. Możesz to zrobić korzystając z atrybutów .dt.year, .dt.month i .dt.day obiektu Series pandas.

- c) Zakoduj daty jako liczby, przypisując każdemu roku wartość 365 (liczba dni w roku), każdemu miesiącu wartość 30 (przybliżona liczba dni w miesiącu) i dodając do tego liczbę dni. Wzór to rok * 365 + (miesiąc - 1) * 30 + dzień.

Przykład szablonu metody:

In [63]:
# tu nic nie robisz, masz to zrobić wewnątrz klasy DateVariable, która jest nieco niżej

def encode_as_number(self) -> pd.Series:
    # Tu wstaw logikę kodowania daty jako liczby.
    return # zakodowana jako liczba pd.Series


W tej klasie musisz napisać 2 metody opisane w poleceniu :

In [64]:
class DateVariable() :

    """
    A class used to represent a date variable from some dataset.
    
    Attributes
    ----------
    column : pandas Series.
        A column from a pandas DataFrame or a standalone column.
    
    Static Methods
    -------
    parse_string_date(string_date : str) -> pd.Timestamp
        Parses a string date into a pandas Timestamp object.
    
    Methods
    -------
    encode_as_number() -> pd.Series
        Encodes the date as a number, where each day is represented by a unique number.
    """
    
    def __init__(self, column : pd.Series) :
        self.column = column
    
    @staticmethod
    def parse_string_date(string_date : str) :

        date = pd.to_datetime( string_date )
        
        return date
        
        
    def encode_as_number(self) :

        years, months, days = 0, 0, 0

        if self.column.dtype == 'datetime64[ns]' :
            pass

        else :
            self.column = self.column.apply(lambda x : DateVariable.parse_string_date(x) if not pd.isnull(x) else x)
    

        years, months, days = self.column.dt.year, self.column.dt.month, self.column.dt.day
            
        
        return years * 365 + (months - 1) * 30 + days
    

 Na koniec zobacz czy metoda encode_as_number poprawnie koduje daty jako liczby.

# Zadanie 3

Klasa NumericVariable jest przeznaczona do reprezentowania zmiennej numerycznej z jakiegoś zbioru danych. Będziecie tworzyć konstruktor oraz metodę detect_outlier_iqr. Oto jak to zrobić:
1. Konstruktor __init__(self, column: pd.Series):
Konstruktor ma za zadanie inicjalizować instancję klasy, przyjmując jedną zmienną: kolumnę danych w formacie pd.Series. Oto jakie kroki należy wykonać:

- a) W definicji konstruktora określ, że przyjmuje on jeden parametr column, który jest obiektem typu pd.Series.

- b) W ciele konstruktora przypisz przekazany argument column do atrybutu instancji o tej samej nazwie column.

2. Metoda detect_outlier_iqr(self) -> list:
Ta metoda ma za zadanie wykrywać obserwacje odstające (outliers) w przekazanej kolumnie numerycznej, korzystając z metody zakresu międzykwartylowego (IQR). Realizacja tej metody powinna obejmować następujące kroki:

- a) Oblicz pierwszy (Q1) i trzeci (Q3) kwartyl danych za pomocą funkcji np.percentile z biblioteki numpy.

- b) Oblicz zakres międzykwartylowy (IQR) jako różnicę między Q3 a Q1.

- c) Określ wartości graniczne dla wykrywania odstających obserwacji: 1.5 * IQR powyżej Q3 i 1.5 * IQR poniżej Q1.

- d) Użyj tych granic do identyfikacji indeksów obserwacji odstających, które są mniejsze niż dolna granica lub większe niż górna granica.

- e) Zwróć listę indeksów tych obserwacji odstających.

Przykład szablonu metody:

In [65]:
# Tu nic nie robisz, masz to zrobić wewnątrz klasy NumericVariable, która jest niżej

def detect_outlier_iqr(self) -> list:
    # Tu wstaw logikę wykrywania obserwacji odstających.
    return # lista indeksów obserwacji odstających


*** Początek wskazówki do podpunktu d) ***

Pamiętaj, że Twój atrybut instancji klasy NumericVariable jest typu pd.Series.

In [66]:
# przykładowy obiekt typu pd.Series
seria = pd.Series([1, 2 ,3])
seria

0    1
1    2
2    3
dtype: int64

In [67]:
# subsetting obiektu pd.Series
# wyświetlamy elementy mniejsze od 3
seria[seria < 3]

0    1
1    2
dtype: int64

In [68]:
# w zadaniu będziemy bazować na indeksach
# poniżej przykład jak znależć indeksy danych mniejszych od 3 i dostać je jako listę
seria[seria < 3].index.tolist()

[0, 1]

W d) musisz zrobić to samo dla górnej i dolnej wartości granicznej a następnie połączyć te 2 listy.

\\\ Koniec wskazówki do podpunktu d) \\\

In [69]:
class NumericVariable() :
    """
    A class used to represent a numeric variable from some dataset.

    Attributes
    ----------
    column : pandas Series.
        A column from a pandas DataFrame or a standalone column.
    
    Methods
    -------
    detect_outlier_iqr() -> list
        Detects outliers using the interquartile range method.
    """

    def __init__(self, column : pd.Series) :
        self.column = column
    
    def detect_outlier_iqr(self) :

        """ 
        Detects outliers using the interquartile range method.
        Returns a list of indices of the outliers in the given column.
        The method uses the 1.5 * IQR rule to detect outliers.


        Returns
        -------
        list
            A list of indices of the outliers in the given column.
        """

        q1 = np.percentile(self.column, 25)
        q3 = np.percentile(self.column, 75)
        span = iqr(self.column)
        too_big = span * 1.5 + q3
        too_small = q1 - span * 1.5
        small_outliers_indices = self.column[self.column < too_small].index.tolist()
        big_outliers_indices = self.column[self.column > too_big].index.tolist()
        outliers_indices = small_outliers_indices + big_outliers_indices

        return outliers_indices

In [70]:
class CategoricalData(DataSet) :
    """ 
    A class used to represent a set of categorical variables from some dataset.
    Inherits from the DataSet class.
    
    Attributes
    ----------
    path : str, optional. Defaults to None.
        The path to the data file.
    data : pandas DataFrame, optional. Defaults to None.
        The data already in DataFrame form.
    max_uniq_vals : int, optional. Defaults to 10.
        The maximum number of unique values a column can have. 
        If a column has more unique values than this, it will not be encoded.
        It is useful for avoiding computational overhead when one-hot encoding.
    
    Methods
    -------
    encode_data(method, show_mapping=False) -> pd.DataFrame
        Encodes the categorical data using the given method.
    
    """

    def __init__(self, path = None, data = None, max_uniq_vals=10) :
        super().__init__(path, data)
        self.cat_data = self.data.select_dtypes(include='object')
        self.unique_values = self.cat_data.nunique()
        self.cols_to_encode = self.unique_values[self.unique_values <= max_uniq_vals].index.tolist()
        self.cat_data = self.data[self.cols_to_encode]
    
    def encode_data(self, method, show_mapping=False) -> pd.DataFrame:
        
        encoded_data = {}
        
        for column in self.cat_data.columns:
            categorical_col = CategoricalVariable(self.cat_data[column])
            encoded_data[column] = categorical_col.encode_data(method, show_mapping)
        
        df = pd.DataFrame()

        for k, v in encoded_data.items():
            if method == 'ordinal':
                df[k] = v
            elif method == 'one_hot':
                df = pd.concat([df, v], axis=1)
            else:
                raise ValueError(f"Encoding method {method} not recognized.")
        
        return df

In [71]:
class NumericData(DataSet) :

    """
    A class used to represent numeric data from some dataset. 
    Inherits from the DataSet class (which can consist of both numeric and categorical data).

    Attributes
    ----------
    path : str, optional. Defaults to None.
        The path to the data file.
    data : pandas DataFrame, optional. Defaults to None.
        The data already in DataFrame form.
    
    Methods
    -------
    detect_outliers(method='iqr', by_column=False) -> list or dict
        Detects outliers using the given method.
        Defaults to the interquartile range method.
    """
    
    def __init__(self, path = None, data=None) :
        
        super().__init__(path, data)
        self.num_data = self.data.select_dtypes(include='number')
    
    def detect_outliers(self, method = 'iqr', by_column = False) :

        """ 
        Detects outliers using the given method.
        By default uses interquartile range method.
        Under the hood, it applies NumericVariable.detect_outlier_iqr() to each numeric column,
        and returns combined data as list / dictionary.
        
        Parameters
        ----------
        method : str, optional. Defaults to 'iqr'.
            The method to use for outlier detection.

        by_column : bool, optional. Defaults to False.
            If True, returns a dictionary of outliers for each column.
            If False, returns a list of indices of the outliers in the dataset (which is a set of all columns).
        
        Returns
        -------
        list or dict
            A list of indices of the outliers in the dataset (if by_column is False).
            A dictionary of outliers for each column (if by_column is True).
        """
            
        outliers = {}
            
        for c in self.num_data.columns :
            
            numeric_col = NumericVariable(self.data[c])
            
            if method == 'iqr' :
                indices_outliers_iterab = numeric_col.detect_outlier_iqr()
                
                if indices_outliers_iterab != [] :
                    outliers[c] = indices_outliers_iterab
                
                
            elif method == 'z_score' :
                pass
            else :
                raise ValueError(f"Outlier detection method {method} not recognized.")
            
        if by_column :
            return outliers

        outlier_indices = []

        for v in outliers.values() :
            outlier_indices += v
            
        return outlier_indices

In [73]:
class DataImputation() :

    """
    A class used to represent data imputation methods - methods used to fill in missing values in a dataset.

    Methods
    -------
    mode_imputation(column : pd.Series) -> pd.Series
        Imputes missing values using the mode of the given column.
        Most suitable for categorical data.
    
    mean_imputation(column : pd.Series) -> pd.Series
        Imputes missing values using the mean of the given column.
        Most suitable for numeric data without outliers.
    
    median_imputation(column : pd.Series) -> pd.Series
        Imputes missing values using the median of the given column.
        Most suitable for numeric data with outliers.
    """

    def mode_imputation(self, column : pd.Series) :
        
        mode = column.mode()[0]
        
        return column.fillna(mode)
    
    def mean_imputation(self, column : pd.Series) :
            
        mean = column.mean()
            
        return column.fillna(mean)

    def median_imputation(self, column : pd.Series) :
            
        median = column.median()
            
        return column.fillna(median)



In [85]:
class PreparingDataset(CategoricalData, NumericData) :

    """ 
    A class used to describe how to prepare data for predictive modelling.
    Inherits from both CategoricalData and NumericData classes.
    
    Attributes
    ----------
    path : str, optional. Defaults to None.
        The path to the data file.
    data : pandas DataFrame, optional. Defaults to None.
        The data already in DataFrame form.
    date_col_name : str, optional. Defaults to None.
        The name of the column that contains date data.
    
    Methods
    -------
    prepare_categoricl_data(method='one_hot', impute_missing=False) -> pd.DataFrame
        Prepares the categorical data for predictive modelling, meaning it
        fills in the missing values with the mode (most common value)
        and encodes values such as 'green' as numbers."""

    def __init__(self, path = None, data = None, date_col_name = None) :
        
        CategoricalData.__init__(self, path = path, data = data)       
        NumericData.__init__(self, path = path, data = data)

        if date_col_name is not None :
            self.date_data = self.data[date_col_name]
    
    def prepare_categoricl_data(self, method = 'one_hot', impute_missing = False) :

        if impute_missing :
            for c in self.cat_data.columns :
                most_common = self.cat_data[c].mode()[0]
                self.cat_data[c] = self.cat_data[c].fillna(most_common)
        
        return self.encode_data(method = method) 

    def prepare_numeric_data(self, method = 'iqr', remove_outliers = False, impute_missing = False) :

        outliers = self.detect_outliers(method)
        outliers_by_column = self.detect_outliers(method, by_column=True)

        if impute_missing :
            for c in self.num_data.columns :
                if c in outliers_by_column.keys() :
                    median = self.num_data[c].median()
                    self.num_data[c] = self.num_data[c].fillna(median)
                else :
                    mean = self.num_data[c].mean()
                    self.num_data[c] = self.num_data[c].fillna(mean)

        if remove_outliers :
            indices_keep = [i for i in self.num_data.index if i not in outliers]
            self.num_data = self.num_data.iloc[indices_keep]

            return self.num_data
        
        else :
            
            return self.num_data

    def prepare_date_data(self) :
        
        return DateVariable(self.date_data).encode_as_number()

    
    
    

In [86]:
class CleanDataset(PreparingDataset) :

    """ 
    A class used to represent a clean dataset which is ready for predictive modelling.

    Attributes
    ----------
    path : str, optional. Defaults to None.
        The path to the data file.
    data : pandas DataFrame, optional. Defaults to None.
        The data already in DataFrame form.
    date_col_name : str, optional. Defaults to None.
        The name of the column that contains date data.
    
    Methods
    -------
    get_data(encoding_method='one_hot', outlier_method='iqr', remove_outliers=True, impute_missing=False) -> pd.DataFrame
        Returns the clean dataset ready for predictive modelling.

    """

    def __init__(self, path = None, data = None, date_col_name = None) :
        super().__init__(path = path, data = data, date_col_name = date_col_name)
        self.date_col_name = date_col_name
    
    def get_data(self, encoding_method = 'one_hot', outlier_method = 'iqr', remove_outliers = True, impute_missing = False) :

        categorical = self.prepare_categoricl_data(method = encoding_method, impute_missing = impute_missing)
        numeric = self.prepare_numeric_data(method=outlier_method, remove_outliers = remove_outliers, impute_missing=impute_missing)

        data_parts = [categorical, numeric]
        
        if self.date_col_name is not None :
            date_calendar = self.prepare_date_data()
            data_parts.append(date_calendar)
        
        return pd.concat(data_parts, axis=1)

In [91]:
df = CleanDataset(data = deng_train, date_col_name = 'week_start_date').get_data(impute_missing=True, remove_outliers=True, encoding_method='one_hot')

C:\Users\Predator\AppData\Local\Temp\ipykernel_13180\2005783920.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.cat_data[c] = self.cat_data[c].fillna(most_common)
C:\Users\Predator\AppData\Local\Temp\ipykernel_13180\2005783920.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.num_data[c] = self.num_data[c].fillna(mean)


In [88]:
df.head()

city  year  weekofyear   ndvi_ne   ndvi_nw   ndvi_se   ndvi_sw  \
0   1.0  1990          18  0.122600  0.103725  0.198483  0.177617   
1   1.0  1990          19  0.169900  0.142175  0.162357  0.155486   
2   1.0  1990          20  0.032250  0.172967  0.157200  0.170843   
3   1.0  1990          21  0.128633  0.245067  0.227557  0.235886   
4   1.0  1990          22  0.196200  0.262200  0.251200  0.247340   

   precipitation_amt_mm  reanalysis_air_temp_k  reanalysis_avg_temp_k  ...  \
0                 12.42             297.572857             297.742857  ...   
1                 22.82             298.211429             298.442857  ...   
2                 34.54             298.781429             298.878571  ...   
3                 15.36             298.987143             299.228571  ...   
4                  7.52             299.518571             299.664286  ...   

   reanalysis_relative_humidity_percent  reanalysis_sat_precip_amt_mm  \
0                             73.365714                         12.42   
1                             77.368571                         22.82   
2                             82.052857                         34.54   
3                             80.337143                         15.36   
4                             80.460000                          7.52   

   reanalysis_specific_humidity_g_per_kg  reanalysis_tdtr_k  \
0                              14.012857           2.628571   
1                              15.372857           2.371429   
2                              16.848571           2.300000   
3                              16.672857           2.428571   
4                              17.210000           3.014286   

   station_avg_temp_c  station_diur_temp_rng_c  station_max_temp_c  \
0           25.442857                 6.900000                29.4   
1           26.714286                 6.371429                31.7   
2           26.714286                 6.485714                32.2   
3           27.471429                 6.771429                33.3   
4           28.942857                 9.371429                35.0   

   station_min_temp_c  station_precip_mm  week_start_date  
0                20.0               16.0           726470  
1                22.2                8.6           726477  
2                22.8               41.4           726484  
3                23.3                4.0           726491  
4                23.9                5.8           726498  

[5 rows x 24 columns]